In [23]:
import scipy.stats as stats
import pandas as pd
import numpy as np

# Вариант 1

Независимые переменные: расход в городе, расход на шоссе, мощность
Зависимая: цена

In [24]:
data = pd.read_csv("cars93.csv")
Y = data['Price']
features = ['Horsepower', 'MPG.city', 'MPG.highway']
X = data[features].copy()
X['Free coeff'] = [1] * len(X)
features.append("Free coeff")

In [25]:
X

,Horsepower,MPG.city,MPG.highway,Free coeff
0,140,25,31,1
1,200,18,25,1
2,172,20,26,1
3,172,19,26,1
4,208,22,30,1
...,...,...,...,...
88,109,17,21,1
89,134,21,30,1
90,178,18,25,1
91,114,21,28,1


In [26]:
A = X.T @ X

In [27]:
A

,Horsepower,MPG.city,MPG.highway,Free coeff
Horsepower,2176206,280948,373151,13376
MPG.city,280948,49426,63101,2080
MPG.highway,373151,63101,81293,2705
Free coeff,13376,2080,2705,93


In [28]:
beta = np.linalg.inv(A) @ X.T @ Y
beta.index = features

In [29]:
beta

Horsepower     0.131314
MPG.city      -0.038600
MPG.highway   -0.178859
Free coeff     6.688679
dtype: float64

In [30]:
y_pred = X @ beta

In [31]:
mse = 0
for i in range(len(X)):
    mse += (y_pred.iloc[i] - Y.iloc[i])**2
    print(y_pred.iloc[i] - Y[i])
mse /= len(X)
print("MSE: ", mse)

2.6629935095446253
-6.114823552501772
-5.247669835299334
-13.809069656541404
-2.213006745550963
-0.9606209166932764
2.4705854254162496
1.536100163339622
-2.850555947646299
-6.837623194985916
0.2371908529677782
0.3292854123456781
2.6870026662205806
6.857447104579018
-0.7643034840604468
7.903478738861434
7.599285995528959
4.9055030368070085
2.9551598346284678
2.599650101235028
3.508083579956562
-8.93051563739241
2.5478116006193012
1.5261610130004009
0.8416752702192962
1.9230088731500175
-1.4197245510227727
15.295418282807983
-0.45218839938069877
9.709793858342135
0.4625101126754725
7.011972676909538
2.3164199418844014
-1.4594728232369913
1.4976065131469305
1.6730093538544963
-1.303747148486206
5.393179499723544
-5.207599883299917
-1.589992123118428
1.427870329977015
-1.86601455258341
1.1015936883025574
2.3033594476983463
6.935489986353748
1.6847535099556357
3.9956629260794028
-9.297168590401093
-2.005672406820171
-3.774260496533799
-11.90763528403022
-7.180543858601997
1.4192387720322657

# Проверим подозрения
- Чем больше мощность тем больше цена - действительно

- Цена изменяется в зависимости от расхода в городе

Действительно получившихся коэффициентов видно, что этот действительно заметно меньше (по модулю) остальных. Для уверенности воспользуемся T-тестом

- $H_0: \beta_1 = 0$
- $H_1: \beta_1 \neq 0$

$$T = \frac{\hat{\beta}_1}{SE(\hat{beta}_1)} \sim T(n - k)$$

$SE(\hat{beta}_1) = \sqrt{Cov(\beta)_{i i}}$

$Cov(\beta) = \sigma^2 (X^T X)^{-1}$

$\sigma^2 = \frac{\sum_{i=1}^n (y_i - \hat{y}_i)^2}{n - k}$


In [36]:
df = len(X) - len(features)
sigma = np.sum((Y - y_pred)**2) / df
se1 = np.sqrt(np.linalg.inv(A)[1, 1] * sigma)

In [33]:
t_stat = beta.iloc[1] / se1
print("T-statistic: ", t_stat)
alpha = 0.05
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df))
print("p-value: ", p_value)
if stats.t.ppf(alpha / 2, df) <= t_stat <= stats.t.ppf(1 - alpha / 2, df):
    print("Accept H0")
else:
    print("Reject H0")

T-statistic:  -0.10809771106483754
p-value:  0.9141615110844346
Accept H0


Коэффициенты при расходе в городе и расходн на шоссе одновременно равны 0
- $H_0 : \beta_1 = \beta_2 = 0$
- $H_1 : \overline{H_0}$

Для этого воспользуемся F-тестом для проверки значимости линейной регрессии

$$F = \frac{SSE_F - SSE_R}{df_R - df_F} \cdot \frac{df_F}{SSE_F} \sim F(k - 2, n - k)$$

- $SSE_F = \sum_1^n (y_i - \hat{y}^F_i)^2$
- $SSE_R = \sum_1^n (y_i - \hat{y}^R_i)^2$

In [34]:
restricted = features[:1] + features[-1:]
X_r = X[restricted]
beta_r = beta[restricted]
y_pred_r = X_r @ beta_r

sse_f = np.sum((y_pred - Y)**2)
sse_r = np.sum((y_pred_r - Y)**2)

n = len(X)
df_f = n - len(features)
df_r = n - (len(features) - len(restricted))

f_stat = (sse_r - sse_f) / (df_r - df_f) * (df_f / sse_f)
print("F statistic: ", f_stat)
print("P-value: ", 1 - stats.f.cdf(f_stat, df_r - df_f, df_f))

if f_stat < stats.f.ppf(0.95, df_r - df_f, df_f):
    print("Accept H0")
else:
    print("Reject H0")


F statistic:  49.66474832533822
P-value:  3.219646771412954e-15
Reject H0
